In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, Subset

import nevergrad as ng

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from python.models import TCN
from python.data import Data, collate_fn
from python.optimizations import (weight_norm, VariationalDropout, 
    VariationalHidDropout, WeightDrop, embedded_dropout)

from python.utils import *

In [2]:
batch_size = 100 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Torch version: {}'.format(torch.__version__))
print('Device: %s' % (device))
if torch.cuda.device_count() > 1: print("Let's use", torch.cuda.device_count(), "GPUs!")

Torch version: 1.0.0
Device: cpu


In [3]:
data = pd.read_csv('../data/data.csv', index_col=0)

In [4]:
np.random.seed(1111)

train_valid_ratio = 0.9

train_indices = np.sort(np.random.choice(data['idx'], int(len(data)*train_valid_ratio), replace=False))
valid_indices = np.sort(np.array(list(set(np.arange(len(data))) - set(train_indices))))

train_set = Data(data.iloc[train_indices])
valid_set = Data(data.iloc[valid_indices])

train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, 
                         drop_last=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset=valid_set, batch_size=len(valid_set), collate_fn=collate_fn)

file = open('logfile.txt','w') 

In [16]:
# dummy function

def train(nhid, levels, kernel_size, dropout, epochs=500):
    file = open('logfile.txt','a') 
    file.write('call train(): {}, {}, {}, {}'.format(nhid, levels, kernel_size, dropout))
    print('call train(): {}, {}, {}, {}'.format(nhid, levels, kernel_size, dropout))
    
    valid_loss_vec = []
    input_size = 120
    output_size = 2
    clip = 0.2
    
    model = TCN(input_size, output_size, nhid, levels, kernel_size, dropout).to(device)

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        
    optimizer = optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999), eps=1e-08)
    
    loss_function = nn.NLLLoss(weight=torch.tensor([1., 70.], dtype=torch.double).to(device))

        
    for epoch in range(epochs): 
        print("Epoch: {:d}".format(epoch), end="\r")
        file.write("Epoch: {:d}".format(epoch))
        
        for i, (feature, label) in enumerate(train_loader):

            # Clear out accumulates gradients 
            model.zero_grad()

            # Forward pass
            out = model(feature.to(device))

            # Backward propagation
            loss = loss_function(out, label.to(device))
            loss.backward()    
            if clip > 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

        with torch.no_grad():
            input, label = next(iter(valid_loader))

            out = model(input.to(device))
            loss = loss_function(out, label.to(device)).item()
            valid_loss_vec.append(loss)
            
            min_idx = np.argmin(valid_loss_vec)
            
        print('            Validation Loss: {:.3}'.format(loss), end="\r")
        file.write('            Validation Loss: {:.3}'.format(loss))
            
        if epoch > min_idx + 30:
            print('\nbreak')
            file.write('break')
            break       
    
    min_idx = np.argmin(valid_loss_vec)
    min_loss = np.min(valid_loss_vec)

    print('Minimum at {}: {}\n'.format(min_idx, min_loss))
    file.write('Minimum at {}: {}\n'.format(min_idx, min_loss))
    file.close()
    return loss

In [18]:
# argument transformation
nhid = ng.var.OrderedDiscrete([70, 80, 90, 100])
levels =  ng.var.OrderedDiscrete([6, 7, 8, 9])
kernel_size = ng.var.OrderedDiscrete([4, 5, 6, 7, 8])
dropout = ng.var.OrderedDiscrete([0.0 , 0.05, 0.1, 0.15, 0.2, 0.3])

budget = 10 

instrum = ng.Instrumentation(nhid, levels, kernel_size, dropout)

optimizer = ng.optimizers.OnePlusOne(instrumentation=instrum, budget=budget)

recommendation = optimizer.optimize(train)

In [74]:
print(recommendation)

Candidate(args=(55, 6, 7, 0.1999999756279673), kwargs={})
